In [12]:
from functions.functions import *
from config import model

In [13]:
def main_workflow(file_path, model, user_question):
    # Step 1: Upload Document
    docs = upload_document(file_path)
    for doc in docs:
        print(doc.page_content)
        print("-------")
    
    # Step 2: Initialize Vector Database
    vector_db = initialize_vector_db(docs)
    
    # Step 3: Retrieve Context for User's Question
    context = retrieve_context(vector_db, user_question)
    print("\n\nCONTEXT RETRIEVED:")
    for part in context:
        print(part)
        print("-------------")
    
    # Step 4: Generate Answer with Gemini API
    answer = generate_response_with_context(model, context, user_question)
    
    return answer


In [14]:
file_path = "documents/docker.txt"
user_question = "How to delete container?"

response = main_workflow(file_path, model, user_question)
print("RESPONSE:")
print(response[0])


CHUNKING OF DOCUMENT: ...
CHUNKING COMPLETE!
Instructions for Using Docker
Docker is a platform to create, deploy, and run applications inside containers. Below is a step-by-step guide:

1. Install Docker
Visit the Docker website and download Docker Desktop for your operating system.
Install Docker and follow the setup wizard.
Verify installation by running in a terminal: docker --version
-------
2. Understand Key Concepts
Image: A template for creating containers (e.g., Ubuntu, Python).
Container: A lightweight, isolated instance of an image.
Dockerfile: A script to automate image creation.
-------
3. Basic Docker Commands
Pull an image: docker pull <image_name>
Run a container: docker run -it <image_name>
List running containers: docker ps
Stop a container: docker stop <container_id>
-------
4. Create a Dockerfile
Write a Dockerfile:
Dockerfile
FROM python:3.9
WORKDIR /app
COPY . .
RUN pip install -r requirements.txt
CMD ["python", "app.py"]
Build and run: docker build -t my_app .
do

In [5]:
from functions.functions import *

search_info_to_txt("What is Monopoly game?", top=5, similarity=0.7)

Processing Root Links: 100%|██████████| 3/3 [00:16<00:00,  5.65s/it]


['https://en.wikipedia.org/wiki/History_of_Monopoly',
 'https://shop.hasbro.com/en-us/product/monopoly-game/7EABAF97-5056-9047-F577-8F4663C79E75',
 'https://www.dicebreaker.com/games/monopoly/how-to/how-to-play-monopoly']

In [ ]:
from pydork.engine import SearchEngine

search_engine = SearchEngine()

search_engine.ENGINE
search_result = search_engine.search('super mario', search_type="text")

c:\Users\raudvee\OneDrive - Webfleet\Desktop\Code-Dev\RAG-Implementation\.venv\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\raudvee\OneDrive - Webfleet\Desktop\Code-Dev\RAG-Implementation\.venv\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'consent.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [16]:
query = "supermario"

In [21]:
import requests
from bs4 import BeautifulSoup
page = requests.get(f"https://www.google.ru/search?q=estonia&newwindow=1&sca_esv=b1ce1c954e88e9aa&ei=6qU8Z5ubFfOJ9u8Pm7K92A8&ved=0ahUKEwibuKzV0uiJAxXzhP0HHRtZD_sQ4dUDCHA&oq=estonia&gs_lp=Egxnd3Mtd2l6LXNlcnAiB2VzdG9uaWEyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyChAAGLADGNYEGEcyDRAAGIAEGLADGEMYigUyDRAAGIAEGLADGEMYigUyDRAAGIAEGLADGEMYigUyDRAAGIAEGLADGEMYigUyDRAAGIAEGLADGEMYigUyDhAAGLADGOQCGNYE2AEBMg4QABiwAxjkAhjWBNgBATIOEAAYsAMY5AIY1gTYAQEyExAuGIAEGLADGEMYyAMYigXYAQEyExAuGIAEGLADGEMYyAMYigXYAQEyExAuGIAEGLADGEMYyAMYigXYAQEyExAuGIAEGLADGEMYyAMYigXYAQFI7A5QAFgAcAJ4AZABAJgBAKABAKoBALgBDMgBAJgCAqACCpgDAIgGAZAGE7oGBggBEAEYCZIHATKgBwA&sclient=gws-wiz-serp")
soup = BeautifulSoup(page.content)
import re
links = soup.findAll("a")
for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
    print(re.split(":(?=http)",link["href"].replace("/url?q=","")))

In [1]:
try:
	from googlesearch import search
except ImportError: 
	print("No module named 'google' found")

# to search
query = "Geeksforgeeks"

def google_search(query: str):
    list_of = []


    for link in search(query=query, num=10, stop=11):
        list_of.append(link)
        
    for link in search(query=query, num=10, stop=3, tbs="qdr:h"):
        list_of.append(link)
    for link in search(query=query, num=10, stop=3, tbs="qdr:d"):
        list_of.append(link)
    for link in search(query=query, num=10, stop=3, tbs="qdr:m"):
        list_of.append(link)
        
    return set(list_of) 




In [2]:
google_search("Who is donald trump?")

{'http://t3.gstatic.com/licensed-image?q=tbn:ANd9GcQrJzSTI4OI2aLe6DgXajqOO5E87Z3nGiTCZcn_-Q9xqZ_CLtLNLeYib4xxZUSGZgzb',
 'https://en.wikipedia.org/wiki/Donald_Trump',
 'https://nl.wikipedia.org/wiki/Donald_Trump',
 'https://trumpwhitehouse.archives.gov/people/donald-j-trump/',
 'https://www.britannica.com/biography/Donald-Trump',
 'https://www.britannica.com/biography/Donald-Trump/Foreign-relations',
 'https://www.britannica.com/biography/New-York-indictment-of-Donald-Trump-2229245',
 'https://www.britannica.com/facts/Donald-Trump',
 'https://www.clickondetroit.com/news/local/2024/11/19/what-is-the-trump-dance-and-where-did-it-come-from/',
 'https://www.donaldjtrump.com/',
 'https://www.ft.com/content/379dadff-92f7-4901-9432-403fd3c56f5d',
 'https://www.independent.co.uk/news/world/americas/us-politics/donald-trump-news-cabinet-picks-live-updates-b2649022.html',
 'https://www.kron4.com/top-stories/ap-top-headlines/ap-trump-and-musk-solidify-their-bond-with-texas-trip-for-rocket-launch/